In [ ]:
%load_ext autoreload
%autoreload 2
import boystats as B
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import numpy as np
import itertools as it
sns.set_context('talk')

In [ ]:
root = 'data/20201028_sicks/'
B.convert(root, 2020)

In [ ]:
df = pd.read_csv(f'{root}/volume.csv')
df['date'] = pd.to_datetime(df.date)
df = df.sort_values('date')
plt.figure(figsize=(14, 8))
df.plot(x='date', y='volume', style='k', alpha=.2, legend=False, ax=plt.gca())
plt.plot(df.date, df.volume.ewm(com=3).mean(), color='k')
plt.ylabel('Volume /ml')
plt.title('Feed volume');

In [ ]:
df = pd.read_csv(f'{root}/sick.csv', dtype=dict(time=str))
df['date'] = pd.to_datetime(df.date)
df['count'] = 1
f = df.groupby('date')['count'].sum()
f = f.reindex(pd.date_range(f.index.min(), f.index.max()), fill_value=0).reset_index().rename(columns=dict(index='date'))

plt.figure(figsize=(14, 8))
plt.plot(f['date'], f['count'].ewm(com=7).mean(), color='k')
f.plot(x='date', y='count', style='k', alpha=.2, legend=False, ax=plt.gca())
plt.ylabel('Daily vomit frequency')
plt.title('Vomits over time')
plt.ylim(f['count'].min(), f['count'].max())

events = pd.read_csv(f'{root}/events.csv', converters=dict(date=pd.to_datetime))
for kind, color, n in zip(sorted(set(events.kind)), ['k', 'b', 'r'], it.count()):
    e = events[events.kind == kind]
    plt.vlines([d.toordinal() for d in e.date], f['count'].min(), f['count'].max(), linestyles='dashed', color=color)
    for _, row in e.iterrows():
        plt.annotate(row.event, (row.date.toordinal(), f['count'].max()), va='top', textcoords='offset points', xytext=(5, -2 - 20*n),
                     fontsize=18, rotation=0, color=color)

In [ ]:
plt.figure(figsize=(14,8))
hours = df.time[~df.time.isna()].apply(lambda x: int(x[:2]))
counts = collections.Counter(hours)
xs = np.arange(0, 24)
plt.gca().bar(xs, [counts.get(x, 0) for x in xs])
plt.gca().set_xticks(xs)
plt.gca().set_xticklabels(['{:02d}'.format(x) for x in xs])
plt.title('Vomits by hour-of-day')

# d = df[df.date >= pd.to_datetime("2020-09-01")]
# hours = d.time[~d.time.isna()].apply(lambda x: int(x[:2]))
# counts = collections.Counter(hours)
# xs = np.arange(0, 24)
# plt.gca().bar(xs, [counts.get(x, 0) for x in xs])
# plt.gca().set_xticks(xs)
# plt.gca().set_xticklabels(['{:02d}'.format(x) for x in xs])
# plt.title('Vomits by hour-of-day')

plt.figure(figsize=(14,8))
days = df.date.apply(lambda x: x.day_name())
counts = collections.Counter(days)
xlabels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
xs = list(range(len(xlabels)))
plt.gca().bar(xs, [counts.get(x, 0) for x in xlabels], width=.3)
plt.gca().set_xticks(xs)
plt.gca().set_xticklabels(xlabels)
plt.title('Vomits by day-of-week');